In [2]:
import pandas as pd
import numpy as np


#Load data
data_training=pd.read_csv('census-income-training.csv',header=0)
data_test=pd.read_csv('census-income-test.csv',header=0)

#Clean data
#replace invalid or missing entries(delete individual row)
# data_training_clean=data_training.replace(regex=[r'\?'],value=np.nan)
# data_training=data_training_clean.dropna(how='any')
# data_test_clean=data_test.replace(regex=[r'\?'],value=np.nan)
# data_test=data_test_clean.dropna(how='any')


# Split the data into features and target label
# drop invalid feature
invalid_feature = 'GRINREG','MIGMTR1','MIGMTR3','MIGSAME','PARENT','PEFNTVTY','PEMNTVTY','PENATVTY'
income_raw_training = data_training['income_morethan_50K']
features_raw_training = data_training.drop(['income_morethan_50K','Id','GRINREG','MIGMTR1','MIGMTR3','MIGSAME','PARENT','PEFNTVTY','PEMNTVTY','PENATVTY'],axis=1)

features_raw_test = data_test.drop(['Id','GRINREG','MIGMTR1','MIGMTR3','MIGSAME','PARENT','PEFNTVTY','PEMNTVTY','PENATVTY'],axis=1)


# Visualize skewed continuous features of original data
#vs.distribution(data)

# Log-transform the skewed features
skewed = ['AHRSPAY','CAPGAIN', 'CAPLOSS','DIVVAL']
features_log_transformed_training = pd.DataFrame(features_raw_training)
features_log_transformed_training[skewed] = features_raw_training[skewed].apply(lambda x: np.log(x + 1))
features_log_transformed_test = pd.DataFrame(features_raw_test)
features_log_transformed_test[skewed] = features_raw_test[skewed].apply(lambda x: np.log(x + 1))

#Normalizing Numerical Features
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() # default=(0, 1)
numerical = ['AAGE', 'ADTOCC', 'AHRSPAY', 'CAPGAIN','CAPLOSS', 'DIVVAL','HHDREL','NOEMP']
features_log_minmax_transform_training = pd.DataFrame(features_log_transformed_training)
features_log_minmax_transform_training[numerical] = scaler.fit_transform(features_log_transformed_training[numerical])
features_log_minmax_transform_test = pd.DataFrame(features_log_transformed_test)
features_log_minmax_transform_test[numerical] = scaler.fit_transform(features_log_transformed_test[numerical])

# Data Preprocessing: One-hot encode the 'features_log_minmax_transform' data using pandas.get_dummies()
features_training_final = pd.get_dummies(features_log_minmax_transform_training)
features_test_final = pd.get_dummies(features_log_minmax_transform_test)
income_training_final= income_raw_training

# Print the number of features after one-hot encoding
#encoded = list(features_final.columns)
#print ' total features after one-hot encoding.'
#display (encoded)

#Split the Data into Training and Test Sets
from sklearn.model_selection import train_test_split
x_train=features_training_final
x_test=features_test_final
y_train=income_training_final


# Xgboost
from xgboost import XGBClassifier
# xgbc=XGBClassifier()
# xgbc.fit(x_train,y_train)
# xgbc_y_predict=xgbc.predict(x_test)
# xgbc_predict = pd.DataFrame(xgbc_y_predict,columns=['prediction_income_morethan_50K'])
# xgbc_predict.to_csv('xgbc_predict.csv')


xgbc=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=4, min_child_weight=3,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.85,
              tree_method='exact', validate_parameters=1, verbosity=None)

xgbc.fit(x_train,y_train)
xgbc_y_predict=xgbc.predict(x_test)
xgbc_predict = pd.DataFrame(xgbc_y_predict,columns=['income_morethan_50K'])
xgbc_predict.to_csv('xgbc_predict.csv')

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:04:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
